<a href="https://colab.research.google.com/github/AlexKressner/Business_Intelligence/blob/main/openai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q cohere tiktoken openai langchain langchain-community langchain-openai pypdf faiss-cpu

In [ ]:
from openai import OpenAI
from langchain_core.documents.base import Document
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from IPython.display import display, HTML
import pypdf
import re
from pprint import pprint

In [ ]:
! git clone https://github.com/AlexKressner/Business_Intelligence

In [ ]:
secret_key = ""

# LangChain
LangChain ist ein Entwicklungsframework für Anwendungen, die auf Sprachmodellen basieren. Es ermöglicht die Erstellung von Anwendungen, die kontextbewusst sind, indem es ein Sprachmodell mit verschiedenen Kontextquellen verbindet, wie Anweisungen, Beispielen und Inhalten, die als Grundlage für Antworten dienen. Zusätzlich unterstützt LangChain die Fähigkeit des Sprachmodells, logisch zu überlegen, um auf Basis des gegebenen Kontexts Entscheidungen zu treffen und entsprechende Aktionen zu initiieren. Weitere Informationen finden Sie unter https://python.langchain.com/docs/get_started/introduction.

# Retrival Augmented Generation
Viele Anwendungen von großen Sprachmodellen (LLMs) erfordern nutzerspezifische Daten, die nicht Teil des Trainingsdatensatzes des Modells sind. Eine aktuell sehr beachtete Methode, um dies zu erreichen, ist durch Retrieval Augmented Generation (RAG). In diesem Prozess werden für den Anwendungskontext relevante Daten abgerufen und dem LLM zur Textgeneration übergeben. Wir werden diese Methode in einem einfachen Beispiel gemeinsam anwenden. Dazu wollen wir eine Anwendung entwickeln, die es dem Nutzer ermöglicht sich über das Europawahlprogramm einer deutschen Partei zu informieren.

<img src="https://python.langchain.com/assets/images/data_connection-95ff2033a8faa5f3ba41376c0f6dd32a.jpg" width=1200 height=400 />

## 1 Document Loader
Mithilfe von sogenannten [Document Loadern](https://api.python.langchain.com/en/latest/community_api_reference.html#module-langchain_community.document_loaders) können Sie Dokumente aus vielen verschiedenen Quellen laden. LangChain bietet über 100 verschiedene Dokumentenlader, mit denen Sie alle Arten von Dokumenten (HTML, PDF, Code) aus allen Arten von Standorten (private S3-Buckets, öffentliche Websites) laden können.

In [ ]:
! ls Business_Intelligence

In [ ]:
DOCUMENT = "Business_Intelligence/Daten/RAG/FDP.pdf"

In [ ]:
loader = PyPDFLoader(DOCUMENT)
doc = loader.load()

In [ ]:
print(doc[5].page_content)

## 2 Text Splitting
Ein wesentlicher Teil von RAG besteht darin, nur die relevanten Teile von Dokumenten abzurufen. Dies umfasst mehrere Transformationsschritte, um die Dokumente für die Wiedergewinnung vorzubereiten. Einer der wichtigsten Schritte hierbei ist das Aufteilen (oder Zerlegen) eines großen Dokuments in kleinere Abschnitte. LangChain bietet mehrere Transformationsalgorithmen dafür, sowie für spezifische Dokumententypen (Code, Markdown usw.) optimierte Logik.

In [ ]:
# helper function
def display_colored_chunks(docs:[str]):
    concatenated_string = ""
    colors = ["red", "blue", "green", "orange", "purple"]  # Specify the font colors

    for i, doc in enumerate(docs):
        chunk = f'<span style="color:{colors[i % len(colors)]}">{doc}</span>'
        concatenated_string += chunk

    # Display the concatenated string
    display(HTML(concatenated_string))

### 2.1 Langchain Text Splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter()
docs = text_splitter.split_documents(doc)

In [ ]:
len(docs)

In [ ]:
display_colored_chunks([doc.page_content for doc in docs[:5]])

Es besteht die Möglichkeit diverse Optionen für einen Text Splitter zu setzen. Zum Beispiel können Sie die Größe der aus einem Split resultierenden Textbausteine (Chunks) definieren und den Umfang der Überlappung zwischen zwei Textbausteinen (Overlap).

<img src="https://miro.medium.com/v2/resize:fit:1400/1*PoFhwSJf5_pa7ZjXG25gtg.png" width=600 height=200 />

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap=0)
docs = text_splitter.split_documents(doc)
len(docs)

In [ ]:
display_colored_chunks([doc.page_content for doc in docs[:5]])

### 2.2 Custom TextSplitter

In [ ]:
# Open the PDF file
with open(DOCUMENT, 'rb') as file:
    # Create a PDF reader object
    pdf_reader = pypdf.PdfReader(file)

    # Initialize an empty string to store the pages
    all_pages = ""

    # Iterate over each page in the PDF
    for page in pdf_reader.pages:
        # Extract the text from the page and append it to the all_pages string
        all_pages += page.extract_text()

In [ ]:
pprint(all_pages[:5000])

In [ ]:
# Aufteilen in Chunks nach einem definierten Muster ("Leerzeichen-Leerzeichen-Neue_Zeile")
chunks = re.split(r"\s\s\n", all_pages)

In [ ]:
docs = [Document(page_content=chunk) for chunk in chunks if len(chunk) > 50]

In [ ]:
len(docs)

In [ ]:
display_colored_chunks([doc.page_content for doc in docs[:10]])

## 3 Text Embedding & Vector Store
Ein weiterer wichtiger Teil des Retrievals ist das Erstellen von Embeddings für Dokumente. Embeddings erfassen die semantische Bedeutung des Textes, was es Ihnen ermöglicht, schnell und effizient andere Textteile zu finden, die ähnlich sind. Nachfolgend finden Sie ein einfaches Embedding von Wörtern als Vektoren. Beispielsweise wird dem Wort `man` der Vektor `[1,7]` und dem Wort `woman` der Vektor `[9,7]` zugeordnet.

<img src="https://www.cs.cmu.edu/~dst/WordEmbeddingDemo/figures/fig5.png" width=650 height=500 />

Mit dem Aufkommen von Embeddings entstand der Bedarf nach Datenbanken, die eine effiziente Speicherung und Suche dieser besonderen Datenstruktur unterstützt. Regelmäßig werden dafür sogenannte Vektordatenbanken verwendet. Wichtige Elemente von Vektordatenbanken sind die **Indexinerung** und das **Abfragen** von in der Datenbank gespeicherten Vektoren.

1. **Indexierung**: Die abzuspeichernden Vektoren sind typischer Weise sehr groß. Aus diesem Grund versucht man, die ursprünglichen Vektoren in einer komprimierten Repräsentation abzubilden. Diese wird dann Vektorindex genannt und erlaubt das schnellere Abfragen von Vektoren.

2. **Abfrage**: Zentrales Element bei der Abfrage von Vektoren einer Vektordatenbank sind sogenannte Ähnlichkeitsmaße. Diese Maße sind die Grundlage dafür, wie eine Vektordatenbank vergleicht und die relevantesten Ergebnisse für eine gegebene Anfrage identifiziert. Ähnlichkeitsmaße sind mathematische Methoden, um zu bestimmen, wie ähnlich zwei Vektoren in einem Vektorraum sind. Sie werden verwendet, um die in der Datenbank gespeicherten Vektoren zu vergleichen und die zu finden, die einem gegebenen Anfragevektor am ähnlichsten sind. Es gibt mehrere Ähnlichkeitsmaße z.B. Kosinusähnlichkeit, euklidische Distanz und Skalarprodukt.

### 3.1 Embeddings

In [ ]:
words = ["my favorite animal is a dog", "i like cats the most", "the soccer game yesterday was great"]
embeddings = OpenAIEmbeddings(api_key=secret_key).embed_documents(words)
len(embeddings), len(embeddings[0])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Calculate cosine distances between elements of the embedding
distances = cosine_similarity(embeddings)

pd.DataFrame(distances, index=words, columns=words)


### 3.2 Vector Store

In [ ]:
db = FAISS.from_documents(docs, OpenAIEmbeddings(api_key=secret_key))

In [ ]:
prompt = "Wie ist die Position der Partei zur europäischen Außenpolitik?"

In [ ]:
chunks = db.similarity_search(prompt, k=2)

In [ ]:
display_colored_chunks([chunk.page_content for chunk in chunks])

## 4 Retrieval
Ein Retriever ist eine Schnittstelle, die Dokumente anhand einer unstrukturierten Anfrage zurückgibt. Es gibt hierbei verschiedene Algorithmen, um geeignete Dokumente zu laden. Wir werden zwei davon betrachten: **Maximum Marginal Relevance** und **Contextual Compression**.

### 4.1 Maximum Marginal Relevance (MMR)
Beim MMR Algorithmus werden zunächst basierend auf semantischer Ähnlichkeit die `fetch_k` ähnlichsten Vektoren gesucht und anschließend die `k` diversesten zurückgebenen.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*AGmzHH_imqwxgMqnss5PLQ.png" width=700 height=300 />

In [ ]:
chunks = db.similarity_search(prompt, k=2)
display_colored_chunks([chunk.page_content for chunk in chunks])

In [ ]:
chunks = db.max_marginal_relevance_search(prompt, k=2, fetch_k=5)
display_colored_chunks([chunk.page_content for chunk in chunks])

### 4.2 Contextual Compression

<img src="https://miro.medium.com/v2/resize:fit:1400/1*HsP17K2tcnVm3H2ififsag.png" width=750 height=480 />

In [ ]:
client = OpenAI(api_key=secret_key)

In [ ]:
def compress_retrieved_text(context_for_compression:str, text_for_compression:str):
    compression_prompt = f"""
    Du erhälst nachfolgend in einfachen Anführungszeichen einen Text. Bitte nehme aus diesem Text nur die Informationen heraus, die für die folgende Frage relevant sind: {context_for_compression}.
    Bitte füge keine Informationen hinzu, die nicht im Text enthalten sind. '{text_for_compression}'
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": compression_prompt}],
        temperature=0.0,
    )
    return response.choices[0].message.content

In [ ]:
chunks = db.similarity_search(prompt, k=5)

In [ ]:
display_colored_chunks([chunk.page_content for chunk in chunks])

In [ ]:
compressed_chunks = []
for chunk in chunks:
    compressed_chunks.append(compress_retrieved_text(prompt, chunk.page_content))

In [ ]:
display_colored_chunks(compressed_chunks)

## 5 Beantwortung der Frage mit Kontext

In [ ]:
text_splitter = RecursiveCharacterTextSplitter()
docs = text_splitter.split_documents(doc)

In [ ]:
len(docs)

In [ ]:
db = FAISS.from_documents(docs, OpenAIEmbeddings(api_key=secret_key))

In [ ]:
prompt = "Wie ist die Position der Partei zur Energiewende?"

In [ ]:
def define_system_message(prompt:str, k:int):
    chunks = db.similarity_search(prompt, k=k)
    context = [chunk.page_content for chunk in chunks]

    system_message = f"""
    Du bist ein freundlicher und hilfsbereiter Assistent. Du erhälst in einfachen Anführungszeichen einen Auszug aus dem Wahlprogramm einer
    politischen Partei für die Europawahl. Du erhälst von einem Nutzer Fragen zu diesem Programmauszug. Deine Aufgabe ist es,
    auf die Frage des Nutzers zu antworten und bei deiner Antwort ausschließlich den Text des bereitsgestellten Wahlprogramms zu verwenden.
    Du darfst keine eigenen Meinungen oder Informationen hinzufügen. Du darfst auch keine Informationen aus anderen Quellen verwenden.
    Bitte antworte sachlich und neutral. Der Textauszug lautet wie folgt: '{" ".join(context)}'
    """

    return system_message

In [ ]:
# Hilfsfunktion zur Interaktion mit der Chat-API
def get_completion(prompt:str, k:int=3, model:str="gpt-3.5-turbo"):
    system_message = [{"role": "system", "content": define_system_message(prompt, k)}] # Wie soll sich das System grundlegend verhalten
    messages = [{"role": "user", "content": prompt}] # Prompt des Nutzers
    response = client.chat.completions.create(
        model=model,
        messages=system_message + messages,
        temperature=0.0,
    )
    return response.choices[0].message.content

In [ ]:
response = get_completion(prompt,1)

In [ ]:
pprint(response)

In [ ]:
response = get_completion(prompt,3)

In [ ]:
pprint(response)

# Aufgabe
1. Welche praktischen Anwendungen sehen Sie in Ihrem Unternehmen für Retrival Augmented Generation (RAG), d.h. welche Geschäftsprozesse lassen sich durch ein Large Language Model in Verbindung mit unternehmensspezifischen Daten verbessern? Bitte nehmen Sie sich etwas Zeit und disktuieren Sie gemeinsam. Wir besprechen einige Ihrer Ideen zum Abschluss gemeinsam!

2. Können Sie Ihre Idee in einem kleinen Prototypen umsetzen? Verwenden Sie den Code dieses Notebooks, d.h.:
  - Speichern Sie ein Dokument/ mehrere Dokumente in Ihren Drive
  - Laden Sie die Daten mit einem geeigneten LangChain Document Loader
  - Splitten Sie die Daten
  - Speichern Sie die Daten in einem Vector Store
  - Erstellen Sie einen Retriever zum Laden der relevanten Informationen aus dem Vector Store
  - Beantworten Sie bespielhaft Fragen
  
  Wenn Sie keinen geeigneten Anwendungsfall oder Dokumente finden, stellen Sie sich folgendes vor: Sie sind ein Versicherungsunternehmen und haben mit Kundinnen verschiedene Kfz-Versicherungen abgeschlossen. Die Versicherungen unterscheiden sich regelmäßig im Versicherungsumfang. Für jeden Kunden liegen Ihnen die Versicherungsunterlagen vor. Sie wollen einen ChatBot entwickeln, an den sich Kundinnen wenden können. Zum Beispiel soll es möglich sein, dass Kundinnnen einen Versicherungsfall melden und der ChatBot Ihnen basierend auf den individuellen Versicherungsdokumenten relevante Informationen bereitstellt.

  Sie finden unter `Business_Intelligence/Daten/RAG/Kfz_Versicherung.pdf` eine Vertragsunterlage. Stellen Sie sich vor, eine Kundin hat einen Steinschlag an der Frontscheibe Ihres Kfz und möchte wissen, ob es sich um einen Voll- oder Teilkaskoschaden handelt. Bitte nutzen Sie die OpenAI-API in Verbindung mit RAG, damit die Kundin eine angemessene Antwort erhält.

In [ ]:
# Hinweise Dokumente im Google-Drive
# 1. Speichern Sie die relevanten Dokumente in Ihren Google-Drive (WICHTIG: Merken Sie sich den Speicherort)
# 2. Verbinden Sie dem Drive mit Ihrem Notebook
# Google-Drive einbinden
from google.colab import drive
drive.mount('/content/drive')
# 3. Definieren Sie den Pfad zum Speicherort
path = "/content/drive/MyDrive/{SPEICHERORT DER DOKUMENTE}"
# 4. Laden Sie die Dokumenten